# SVI Part II: 条件独立, 子采样和 Amortization

> **The Goal: Scaling SVI to Large Datasets**

一般情况下 SVI 过程中每次更新的计算复杂度是正比于样本数，所以我们需要是用 mini-batch 的办法减少复杂度。For a model with $N$ observations, running the `model` and `guide` and constructing the ELBO involves evaluating log pdf's whose 计算复杂度随着样本数 $N$ 比例增长. This is a problem if we want to scale to large datasets. 幸运的是，ELBO 目标自然的支持子采样 provided that 我们的 model/guide 具有一些条件独立性结构 that we can take advantage of. 例如, in the case that the observations are conditionally independent given the latents, the log likelihood term in the ELBO can be approximated with

$$ E[\log p({\bf x}| {\bf z})] \approx \frac{1}{N}\sum_{i=1}^N \log p({\bf x}_i | {\bf z}) \approx  \frac{1}{M}
\sum_{i\in{\mathcal{I}_M}} \log p({\bf x}_i | {\bf z})  $$

where $\mathcal{I}_M$ is a mini-batch of indices of size $M$ with $M<N$ (for a discussion please see references [1,2]). 
很好，问题解决了！但是我们如何在Pyro中实现这一点？

## 在Pyro中标记条件独立


If a user wants to do this sort of thing in Pyro, 则首先需要确保 the model and guide are written in such a way that Pyro can leverage the relevant conditional independencies. Let's see how this is done. Pyro 提供了两种用于标记条件独立性的语言原语(language primitive): `plate` and `markov`. Let's start with the simpler of the two.


<big> `Pyro.plate`: 从 Sequential `plate` 到 Vectorized `plate` </big>

让我们回到 [previous tutorial](svi_part_i.ipynb) 中使用的例子。为了方便起见，让我们在这里回顾 `model` 的主要逻辑：

```python
def model(data):
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data using pyro.sample with the obs keyword argument
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])
```        

For this model the observations are conditionally independent given the latent random variable `latent_fairness`. To explicitly mark this in Pyro we basically just need to replace the Python builtin `range` with the Pyro construct `plate`:

```python
# 我们通过 plate 来声明给定潜变量，观测样本之间的条件独立性。
def model(data):
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data [WE ONLY CHANGE THE NEXT LINE]
    for i in pyro.plate("data_loop", len(data)):  
        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])
```

我们看到 `pyro.plate` 与 `range` 非常相似，但有一个关键区别：each invocation of `plate` requires the user to provide **a unique name.** The second argument is an integer just like for `range`. 

到目前为止，一切都很好。Pyro现在可以利用给定潜在随机变量下观测值的条件独立性。 But how does this actually work? 基本上，`pyro.plate`是使用上下文管理器(context manager)实现的。 At every execution of the body of the `for` loop we enter a new (conditional) independence context which is then exited at the end of the `for` loop body. Let's be very explicit about this: 

- because each observed `pyro.sample` statement occurs within a different execution of the body of the `for` loop, Pyro marks each observation as independent
- this independence is properly a _conditional_ independence _given_ `latent_fairness` because `latent_fairness` is sampled _outside_ of the context of `data_loop`.

在继续之前，让我们提一下使用 sequential `plate` 时要避免的一些陷阱。考虑上述代码片段的以下变体：

```python
# WARNING do not do this!
my_reified_list = list(pyro.plate("data_loop", len(data)))
for i in my_reified_list:  
    pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])
```

这将无法得到想要的效果, since `list()` will enter and exit the `data_loop` context completely before a single `pyro.sample` statement is called. Similarly, the user needs to take care not to leak mutable computations across the boundary of the context manager, as this may lead to subtle bugs. For example, `pyro.plate`  不适用于时序模型，在该模型中循环的 each iteration 都依赖于 previous iteration; in this case a `range` or `pyro.markov` should be used instead.

<big>  `plate`  向量化 </big>

概念上 vectorized `plate` 和 sequential `plate` 是一样的 except that it is a vectorized operation (as `torch.arange` is to `range`). 因此，它有可能实现大幅提速 compared to the explicit `for` loop that appears with sequential `plate`. Let's see how this looks for our running example. First we need `data` to be in the form of a tensor:

```python
data = torch.zeros(10)
data[0:6] = torch.ones(6)  # 6 heads and 4 tails
```

Then we have:

```python
# 向量化 plate 能够帮助加速后续相关计算。
with plate('observe_data'):
    pyro.sample('obs', dist.Bernoulli(f), obs=data)
```

Let's compare this to the analogous sequential `plate` usage point-by-point:

让我们将其与 sequential `plate` 用法进行 point-by-point 比较：
- 这两种模式都要求用户指定一个唯一的 name。
- note that this code snippet 只引入一个(observed)随机变量(namely `obs`), since the entire tensor is considered at once. 
- since there is no need for an iterator in this case, 无需指定 `plate` context 所涉及的张量的长度。



## 子采样

<big> 对于大数据集，每次训练只能用小部分样本进行训练，也就是 subsampling. </big> 

现在，我们知道了如何在Pyro中标记条件独立性。这本身就很有用（请参见SVI第III部分中的 [dependency tracking section](svi_part_iii.ipynb)），但是我们也想进行子采样，以便可以对大型数据集进行 SVI 。根据 model and guide 的结构，Pyro支持几种进行子采样的方法。让我们一一讲解。

### Automatic subsampling with `plate`

Let's look at the simplest case first, in which we get subsampling for free with one or two additional arguments to `plate`:

首先让我们看一下最简单的情况，在这种情况下，we get subsampling for free with one or two additional arguments to `plate`:

```python
for i in pyro.plate("data_loop", len(data), subsample_size=5):
    pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])
```    

That's all there is to it: we just use the argument `subsample_size`. Whenever we run `model()` we now only evaluate the log likelihood for `data` 5个随机抽取的样本; in addition, the log likelihood will be automatically scaled by the appropriate factor of $\tfrac{10}{5} = 2$. 对于向量化 `plate`? 使用方法也完全类似:

```python
with plate('observe_data', size=10, subsample_size=5) as ind:
    pyro.sample('obs', dist.Bernoulli(f), 
                obs=data.index_select(0, ind))
```

重要的是，`plate`现在返回一个索引`ind`的张量，在这种情况下，它的长度为5。请注意，除了参数`subsample_size`外，我们还传递了参数`size`，以便`plate`为注意张量 `data` 的完整大小，以便它可以计算正确的缩放因子。就像sequential `plate` 一样，the user is responsible for selecting the correct datapoints using the indices provided by `plate`. 

最后, note that the user must pass a `device` argument to `plate` if `data` is on the GPU.

### Custom subsampling strategies with `plate`

Every time the above `model()` is run `plate` will sample new subsample indices. Since this subsampling is stateless, this can lead to some problems: basically for a sufficiently large dataset even after a large number of iterations there's a nonnegligible  probability that some of the datapoints will have never been selected. To avoid this the user can take control of subsampling by making use of the `subsample` argument to  `plate`. See [the docs](http://docs.pyro.ai/en/dev/primitives.html#pyro.plate) for details.

Every time the above `model()` is run `plate` will sample new subsample indices. 由于这种子采样是 stateless，因此可能会导致一些问题：对于足够大的数据集，即使经过大量的迭代，也存在不可忽略的可能性，即从未选择某些数据点。为了避免这种情况，the user can take control of subsampling by making use of the `subsample` argument to  `plate`. See [the docs](http://docs.pyro.ai/en/dev/primitives.html#pyro.plate) for details.

### Subsampling when there are only local random variables 

仅存在局部随机变量时进行子采样有所不同。

We have in mind a model with a joint probability density given by

$$ p({\bf x}, {\bf z}) = \prod_{i=1}^N p({\bf x}_i | {\bf z}_i) p({\bf z}_i)  $$

For a model with this dependency structure the scale factor introduced by subsampling scales all the terms in the ELBO by the same amount. 例如，vanilla VAE 就是这种情况。 这就解释了为什么对于VAE，user 可以完全控制子采样并将 mini-batches 直接传递给 model and guide; `plate` is still used, but `subsample_size` and `subsample` are not. To see how this looks in detail, see the [VAE tutorial](vae.ipynb).

### Subsampling when there are both global and local random variables

In the coin flip examples above `plate` appeared in the model but not in the guide, since the only thing being subsampled was the observations. 让我们看一个更复杂的例子 where subsampling appears in both the model and guide. To make things simple let's keep the discussion somewhat abstract and avoid writing a complete model and guide. 

Consider the model specified by the following joint distribution:

$$ p({\bf x}, {\bf z}, \beta) = p(\beta) 
\prod_{i=1}^N p({\bf x}_i | {\bf z}_i) p({\bf z}_i | \beta)  $$

There are $N$ observations $\{ {\bf x}_i \}$ and $N$ local latent random variables 
$\{ {\bf z}_i \}$. There is also a global latent random variable $\beta$. Our guide will be factorized as

$$ q({\bf z}, \beta) = q(\beta) \prod_{i=1}^N q({\bf z}_i | \beta, \lambda_i)  $$

Here we've been explicit about introducing $N$ local variational parameters 
$\{\lambda_i \}$, while the other variational parameters are left implicit. Both the model and guide have conditional independencies. In particular, on the model side, given the $\{ {\bf z}_i \}$ the observations $\{ {\bf x}_i \}$ are independent. In addition, given $\beta$ the latent random variables  $\{\bf {z}_i \}$ are independent. On the guide side, given the variational parameters $\{\lambda_i \}$ and $\beta$ the latent random variables  $\{\bf {z}_i \}$ are independent. To mark these conditional independencies in Pyro and do subsampling we need to make use of `plate` in _both_ the model _and_ the guide. Let's sketch out the basic logic using sequential `plate` (a more complete piece of code would include `pyro.param` statements, etc.). First, the model:

```python
def model(data):
    beta = pyro.sample("beta", ...) # sample the global RV
    for i in pyro.plate("locals", len(data)):
        z_i = pyro.sample("z_{}".format(i), ...)
        # compute the parameter used to define the observation 
        # likelihood using the local random variable
        theta_i = compute_something(z_i) 
        pyro.sample("obs_{}".format(i), dist.MyDist(theta_i), obs=data[i])
```

Note that in contrast to our running coin flip example, here we have `pyro.sample` statements both inside and outside of the `plate` loop. Next the guide:

```python
def guide(data):
    beta = pyro.sample("beta", ...) # sample the global RV
    for i in pyro.plate("locals", len(data), subsample_size=5):
        # sample the local RVs
        pyro.sample("z_{}".format(i), ..., lambda_i)
```

Note that crucially the indices will only be subsampled once in the guide; the Pyro backend makes sure that the same set of indices are used during execution of the model. For this reason `subsample_size` only needs to be specified in the guide.

## Amortization

<big> 变分自编码器(VAE)的由来</big>

Let's again consider a model with global and local latent random variables and local variational parameters:

$$ p({\bf x}, {\bf z}, \beta) = p(\beta) 
\prod_{i=1}^N p({\bf x}_i | {\bf z}_i) p({\bf z}_i | \beta)  \qquad \qquad
q({\bf z}, \beta) = q(\beta) \prod_{i=1}^N q({\bf z}_i | \beta, \lambda_i)  $$

For small to medium-sized $N$ using local variational parameters like this can be a good approach. If $N$ is large, however, the fact that the space we're doing optimization over grows with $N$ can be a real problem. <big> One way to avoid this nasty growth with the size of the dataset is *amortization*.</big>

This works as follows. Instead of introducing local variational parameters, we're going to learn a single parametric function $f(\cdot)$ and work with a variational distribution that has the form 

$$q(\beta) \prod_{n=1}^N q({\bf z}_i | f({\bf x}_i))$$

The function $f(\cdot)$&mdash;which basically maps a given observation to a set of variational parameters tailored to that datapoint&mdash;will need to be sufficiently rich to capture the posterior accurately, but now we can handle large datasets without having to introduce an obscene number of variational parameters. 
This approach has other benefits too: for example, during learning $f(\cdot)$ effectively allows us to share statistical power among different datapoints. Note that this is precisely the approach used in the [VAE](vae.ipynb).

## More about `plate`  

<small> Tensor shapes and vectorized `plate` </small> 

The usage of `pyro.plate` in this tutorial was limited to relatively simple cases. For example, none of the `plate`s were nested inside of other `plate`s. In order to make full use of `plate`, the user must be careful to use Pyro's tensor shape semantics. For a discussion see the [tensor shapes tutorial](tensor_shapes.ipynb).

## 参考文献

[1] `Stochastic Variational Inference`,
<br/>&nbsp;&nbsp;&nbsp;&nbsp;
Matthew D. Hoffman, David M. Blei, Chong Wang, John Paisley

[2] `Auto-Encoding Variational Bayes`,<br/>&nbsp;&nbsp;&nbsp;&nbsp;
Diederik P Kingma, Max Welling